# 26. 분류

## 26.3 MLlib 분류

### 26.3.1 모델 확장성

|           모델           	| 최대 특징 수 	| 최대 학습 데이터 수 	|       최대 타킷 범주 수       	|
|:------------------------:	|:------------:	|:-------------------:	|:-----------------------------:	|
| 로지스틱 회귀            	| 1 ~ 천만 개  	| 제한없음            	| 특징 수 * 클래스 수 < 1천만   	|
| 의사결정 트리            	| 1,000        	| 제한없음            	| 특징 수 * 클래스 수 < 10,000  	|
| 랜덤 포레스트            	| 10,000       	| 제한없음            	| 특징 수 * 클래스 수 < 100,000 	|
| 그래디언트 부스티드 트리 	| 1,000        	| 제한없음            	| 특징 수 * 클래스 수 < 10,000  	|

In [1]:
# 세션 생성
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("classification examples") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark.conf.set('spark.sql.shuffle.partitions', 5)

In [2]:
""" 실습 데이터 로드 """
bInput = spark.read.format("parquet").load("../BookSamples/data/binary-classification")\
    .selectExpr("features", "cast(label as double) as label")

bInput.printSchema()
bInput.show()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)

+--------------+-----+
|      features|label|
+--------------+-----+
|[3.0,10.1,3.0]|  1.0|
|[1.0,0.1,-1.0]|  0.0|
|[1.0,0.1,-1.0]|  0.0|
| [2.0,1.1,1.0]|  1.0|
| [2.0,1.1,1.0]|  1.0|
+--------------+-----+



## 26.4 로지스틱 회귀
+ 가중치는 특징의 중요성을 잘 나타내기 때문에 유용하게 사용됨
+ 정규화 수행 가정

### 26.4.1 모델 하이퍼파라미터
+ family: 해당 모델의 클래스가 다항 혹은 이항인지 구분
+ elasticNetParam: L1, L2의 가중치
+ fitIntercept: 절편이나 임의의 수를 적합시킬지 결정, 정규화하지 않았다면 절편에 적합하는 것이 일반적
+ regParam: 목적 함수에서 일반화 항에 얼마만큼의 가중치를 주는지 결정
+ standardiztion: 입력을 모델로 전달하기 전에 표준화할지 여부를 결정

### 26.4.2 학습 파라미터

+ maxIter: 총 학습 반복 횟수, 기본값은 100
+ tol: 가중치가 충분히 최적화되었음을 나타내는 임곗값, 기본값은 1.0E-6
+ weightCol: 특정 로우에 가중치를 부여하는데 사용하는 가중치 컬럼의 이름

### 26.4.3 예측 파라미터
+ threshold: 0에서 1 범위의 Double 타입의 값을 가지며, 주어진 클래스를 예측하는 확률 임곗값임
+ thresholds: 다중 분류 시 각 클래스에 대한 임곗값 배열

In [3]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()
lrModel = lr.fit(bInput)

print (lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal wi

In [4]:
print(lrModel.coefficients) # 다항 모델은 lrModel.coefficientMatrix, lrModel.interceptVector로 확인
print(lrModel.intercept)

[6.848741326855034,0.35356589010197487,14.814900276915889]
-10.225695864480993


In [7]:
summary = lrModel.summary
print(summary.areaUnderROC)
summary.roc.show(10, 0)
summary.pr.show(10, 0)

1.0
+---+------------------+
|FPR|TPR               |
+---+------------------+
|0.0|0.0               |
|0.0|0.3333333333333333|
|0.0|1.0               |
|1.0|1.0               |
|1.0|1.0               |
+---+------------------+

+------------------+---------+
|recall            |precision|
+------------------+---------+
|0.0               |1.0      |
|0.3333333333333333|1.0      |
|1.0               |1.0      |
|1.0               |0.6      |
+------------------+---------+



## 26.5 의사결정트리

### 26.5.1 모델 하이퍼파라미터
+ maxDepth
+ maxBins: 연속형 특징을 범주형 특징으로 변환할 때 생성해야하는 구간 수를 결정
+ impurity: entropy, gini 중 하나를 설정
+ minInfoGain: 의사결정트리의 분할에 사용할 수 있는 최소 정보 획득을 결정. 값이 클수록 과적합을 방지할 수 있음(기본값 0)
+ minInstancePerNode: 모델이 학습해야 할 최소 인스턴스 수. 깊이를 제한하거나 어떤 말단 노드에서 멈춰야 하는지 결정하며 값이 클수록 과적합 방지(기본값 1)

### 26.5.2 학습 파라미터

+ checkpointInterval: 학습 과정 동안 진행되는 모델의 작업 내용을 저장하는 방법. 특정 노드가 다양한 이유로 충돌할 경우에도 이전까지 진행된 작업 내용을 복구 가능하며 -1로 기능을 끌 수 있음

### 26.5.3 예측 파라미터
+ 임계값만 존재

In [6]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dtModel = dt.fit(bInput)

print(dt.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name. (default: label)
maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree.

### 26.6 랜덤 포레스트와 그래디언트 부스디트 트리

### 26.6.1 모델 하이퍼파라미터
#### 랜덤 포레스트
+ numTrees
+ featureSubsetStrategy: 파라미터 분할 시 고려해야 할 특징 수를 결정(auto, all, sqrt, log2)

#### 그래이언트 부스티드 트리
+ lossType: 손실함수 지정(logistic loss만 지원하는지 확인 필요)
+ maxIter
+ stepSize: 알고리즘 학습률(기본값은 0.1)

### 26.6.2 학습 파라미터

+ checkpointInterval: 학습 과정 동안 진행되는 모델의 작업 내용을 저장하는 방법. 특정 노드가 다양한 이유로 충돌할 경우에도 이전까지 진행된 작업 내용을 복구 가능하며 -1로 기능을 끌 수 있음

### 26.6.3 예측 파라미터
+ 의사결정 나무와 동일

In [7]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier()
rfModel = rf.fit(bInput)

print(rf.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 

In [8]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier()
gbtModel = gbt.fit(bInput)

print(gbt.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 

## 26.7 나이브 베이즈
+ 모든 입력 특징이 음수가 아니어야 함
+ 모델 하이퍼파라미터
    + modelType: bernouli, multinormial
    + weightCol: 각 데이터값에 다른 가중치 설정
+ 학습 파라미터
    + smoothing: additive smoothing, 과적합을 피할 수 있음(기본값 1)

In [9]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes()
nbModel = nb.fit(bInput.where("label != 0"))

print(nb.explainParams())

featuresCol: features column name. (default: features)
labelCol: label column name. (default: label)
modelType: The model type which is a string (case-sensitive). Supported options: multinomial (default) and bernoulli. (default: multinomial)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
smoothing: The smoothing parameter, should be >= 0, default is 1.0 (default: 1.0)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is 

## 26.8 분류와 자동 모델 튜닝을 위한 평가기
+ 이진 분류: BinaryClassificationEvaluator
    + areUnderRoc, areUnderPR 지원
+ 다중클래스 분류: MultiClassificationEvaluator
    + f1, weightedPrecision, weightedRecall, accuracy 지원

In [11]:
# 최근 API 확인 필요

from pyspark.mllib.evaluation import BinaryClassificationMetrics

out = rfModel.transform(bInput)\
    .select("prediction", "label")\
    .rdd.map(lambda x:(float(x[0]), float(x[1])))
metrics = BinaryClassificationMetrics(out)

In [12]:
print(metrics.areaUnderPR)
print(metrics.areaUnderROC)

1.0
1.0


In [13]:
out.toDF().show()

+---+---+
| _1| _2|
+---+---+
|1.0|1.0|
|0.0|0.0|
|0.0|0.0|
|1.0|1.0|
|1.0|1.0|
+---+---+

